In [1]:
!nvidia-smi

Mon Dec 23 11:29:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install gdown

In [3]:
! rm -rf video.mp4

In [4]:
import gdown

# Update with your file's specific ID
file_id = "1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q"
url = f"https://drive.google.com/uc?id={file_id}"

output = "vehicle-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q
To: /kaggle/working/vehicle-counting.mp4
100%|██████████| 35.3M/35.3M [00:00<00:00, 185MB/s]


'vehicle-counting.mp4'

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [6]:
SOURCE_VIDEO_PATH = "/kaggle/working/vehicle-counting.mp4"

In [7]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6037.7/8062.4 GB disk)


In [8]:
# settings
MODEL = "yolov8x.pt"

In [9]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:01<00:00, 127MB/s]  


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [10]:
import json
from ultralytics import YOLO
import cv2
import os

output_video_path = "output_video_test.mp4"
frames_output_dir = "frames_with_bounding_boxes"
json_output_path = "frames_data.json"

# Create frames output directory if not exists
os.makedirs(frames_output_dir, exist_ok=True)

# Open input video
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Prepare data storage for JSON
frames_data = []

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f"Processing frame {frame_count}...")

    # Run YOLOv8 inference on the frame
    results = model.predict(source=frame, conf=0.5, save=False, verbose=False)

    # Data for the current frame
    frame_data = {
        "frame": frame_count,
        "objects": []
    }

    # Draw bounding boxes and labels on the frame
    annotated_frame = frame.copy()
    for result in results[0].boxes:
        box = result.xyxy[0].cpu().numpy()  # Bounding box coordinates: [x_min, y_min, x_max, y_max]
        confidence = float(result.conf[0].cpu().numpy())  # Confidence score
        cls = int(result.cls[0].cpu().numpy())  # Class index
        label = model.names[cls]  # Class label

        # Add object data to frame data
        frame_data["objects"].append({
            "class": label,
            "confidence": confidence,
            "bounding_box": {
                "x_min": int(box[0]),
                "y_min": int(box[1]),
                "x_max": int(box[2]),
                "y_max": int(box[3]),
            }
        })

        # Draw bounding box
        cv2.rectangle(annotated_frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
        # Put label and confidence
        cv2.putText(annotated_frame, f"{label} {confidence:.2f}", (int(box[0]), int(box[1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the frame data to JSON
    frames_data.append(frame_data)

    # Save the annotated frame as an image
    frame_filename = os.path.join(frames_output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, annotated_frame)

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Save all frame data to a JSON file
with open(json_output_path, "w") as json_file:
    json.dump(frames_data, json_file, indent=4)

# Release resources
cap.release()
out.release()
print("Processing complete. Output video, frames, and JSON data saved.")


Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Processing frame 30...
Processing frame 31...
Processing frame 32...
Processing frame 33...
Processing frame 34...
Processing frame 35...
Processing frame 36...
Processing frame 37...
Processing frame 38...
Processing frame 39...
Processing frame 40...
Processing frame 41...
Processing frame 42...
Processing frame 43...
Processing frame 44.

In [12]:
!pip install supervision==0.3.0

In [26]:
import json
import numpy as np
from scipy.spatial.distance import cdist

# Load the frames data from the JSON file
with open('frames_data.json', 'r') as f:
    frames_data = json.load(f)

# Initialize a dictionary to store the vehicle data
vehicles = {}

# Helper function to calculate the center of the bounding box
def get_center(bbox):
    x_min, y_min, x_max, y_max = bbox
    return (x_min + x_max) / 2, (y_min + y_max) / 2

# Initialize a variable for the next tracker ID
next_tracker_id = 1

# Iterate through each frame and assign IDs to vehicles
for frame_data in frames_data:
    frame = frame_data["frame"]
    objects = frame_data["objects"]
    
    for obj in objects:
        bbox = [obj["bounding_box"]["x_min"], obj["bounding_box"]["y_min"],
                obj["bounding_box"]["x_max"], obj["bounding_box"]["y_max"]]
        obj_class = obj["class"]
        confidence = obj["confidence"]
        
        # Calculate the center of the current object's bounding box
        center = get_center(bbox)
        
        # Try to match this object with an existing vehicle
        matched = False
        
        for vehicle_id, vehicle_data in vehicles.items():
            last_frame_data = vehicle_data["last_frame"]
            last_center = vehicle_data["center"]
            
            # If the vehicle was tracked in the previous frame, calculate the distance between the centers
            if last_frame_data == frame - 1:
                distance = np.linalg.norm(np.array(center) - np.array(last_center))
                
                # If the distance is small, assume it's the same vehicle
                if distance < 100:  # You can adjust this threshold as needed
                    matched = True
                    # Update the vehicle's data with new frame information
                    vehicle_data["center"] = center
                    vehicle_data["last_frame"] = frame
                    vehicle_data["bounding_box"] = bbox
                    break
        
        if not matched:
            # If no match, assign a new tracker ID
            vehicles[next_tracker_id] = {
                "tracker_id": next_tracker_id,
                "class": obj_class,
                "confidence": confidence,
                "bounding_box": bbox,
                "center": center,
                "last_frame": frame
            }
            # Increment the next available tracker ID
            next_tracker_id += 1
    
    # Now add the tracker IDs to the objects in the current frame
    for obj in objects:
        bbox = [obj["bounding_box"]["x_min"], obj["bounding_box"]["y_min"],
                obj["bounding_box"]["x_max"], obj["bounding_box"]["y_max"]]
        
        # Find the vehicle that corresponds to this bounding box
        for vehicle_id, vehicle_data in vehicles.items():
            if np.array_equal(vehicle_data["bounding_box"], bbox):
                obj["tracker_id"] = vehicle_data["tracker_id"]
                break

# Save the modified frames data to a new JSON file
with open('frames_data_with_ids.json', 'w') as f:
    json.dump(frames_data, f, indent=4)
